In [0]:
import pandas as pd
import numpy as np


In [0]:
df_train= pd.read_csv('/content/drive/My Drive/Projects/atis-airlinetravelinformationsystem/atis_intents.csv',names=['target','text'])
df_test= pd.read_csv('/content/drive/My Drive/Projects/atis-airlinetravelinformationsystem/atis_intents_test.csv',names=['target','text'])

In [77]:
df_train.head()

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [78]:
df_train.groupby('target').count()

,text
target,
atis_abbreviation,147
atis_aircraft,81
atis_aircraft#atis_flight#atis_flight_no,1
atis_airfare,423
atis_airfare#atis_flight_time,1
atis_airline,157
atis_airline#atis_flight_no,2
atis_airport,20
atis_capacity,16


In [0]:
train_data= df_train.append(df_train.loc[df_train.target.isin(["atis_flight_time", "atis_quantity","atis_flight","atis_cheapest"]), :])
test_data= df_test.append(df_test.loc[df_test.target.isin(["atis_flight_time", "atis_quantity","atis_flight","atis_cheapest"]), :])

In [80]:
train_data.head()

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [0]:
from sklearn.preprocessing import OneHotEncoder as OHE

y_encoder= OHE().fit(np.array(train_data.target).reshape(-1,1))
ytr_encoded= y_encoder.transform(np.array(train_data.target).reshape(-1,1)).toarray()
yts_encoded= y_encoder.transform(np.array(test_data.target).reshape(-1,1)).toarray()

In [82]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
train_data["lower_text"]= train_data.text.map(lambda x: x.lower())
test_data["lower_text"]= test_data.text.map(lambda x: x.lower())

In [0]:
from nltk import word_tokenize

train_data["tokenized"]= train_data.lower_text.map(word_tokenize)
test_data["tokenized"]= test_data.lower_text.map(word_tokenize)

In [85]:
train_data.head()

,target,text,lower_text,tokenized
0,atis_flight,i want to fly from boston at 838 am and arriv...,i want to fly from boston at 838 am and arriv...,"[i, want, to, fly, from, boston, at, 838, am, ..."
1,atis_flight,what flights are available from pittsburgh to...,what flights are available from pittsburgh to...,"[what, flights, are, available, from, pittsbur..."
2,atis_flight_time,what is the arrival time in san francisco for...,what is the arrival time in san francisco for...,"[what, is, the, arrival, time, in, san, franci..."
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfare from tacoma to orlando,"[cheapest, airfare, from, tacoma, to, orlando]"
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fares from pittsburgh to philadelp...,"[round, trip, fares, from, pittsburgh, to, phi..."


In [86]:
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')

def remove_stop(strings, stop_list):
    classed= [s for s in strings if s not in stop_list]
    return classed

stop= stopwords.words("english")
stop_punc= list(set(punctuation))+ stop

train_data["selected"]= train_data.tokenized.map(lambda df: remove_stop(df, stop_punc))
test_data["selected"]= test_data.tokenized.map(lambda df: remove_stop(df, stop_punc))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from nltk.stem import SnowballStemmer

def normalize(text):
    return " ".join(text)

stemmer= SnowballStemmer('english')

train_data["stemmed"]= train_data.selected.map(lambda xs: [stemmer.stem(x) for x in xs])
train_data["normalized"]= train_data.stemmed.apply(normalize)

test_data["stemmed"]= test_data.selected.map(lambda xs: [stemmer.stem(x) for x in xs])
test_data["normalized"]= test_data.stemmed.apply(normalize)

In [88]:
train_data.head()

,target,text,lower_text,tokenized,selected,stemmed,normalized
0,atis_flight,i want to fly from boston at 838 am and arriv...,i want to fly from boston at 838 am and arriv...,"[i, want, to, fly, from, boston, at, 838, am, ...","[want, fly, boston, 838, arrive, denver, 1110,...","[want, fli, boston, 838, arriv, denver, 1110, ...",want fli boston 838 arriv denver 1110 morn
1,atis_flight,what flights are available from pittsburgh to...,what flights are available from pittsburgh to...,"[what, flights, are, available, from, pittsbur...","[flights, available, pittsburgh, baltimore, th...","[flight, avail, pittsburgh, baltimor, thursday...",flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,what is the arrival time in san francisco for...,what is the arrival time in san francisco for...,"[what, is, the, arrival, time, in, san, franci...","[arrival, time, san, francisco, 755, flight, l...","[arriv, time, san, francisco, 755, flight, lea...",arriv time san francisco 755 flight leav washi...
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfare from tacoma to orlando,"[cheapest, airfare, from, tacoma, to, orlando]","[cheapest, airfare, tacoma, orlando]","[cheapest, airfar, tacoma, orlando]",cheapest airfar tacoma orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fares from pittsburgh to philadelp...,"[round, trip, fares, from, pittsburgh, to, phi...","[round, trip, fares, pittsburgh, philadelphia,...","[round, trip, fare, pittsburgh, philadelphia, ...",round trip fare pittsburgh philadelphia 1000 d...


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer= Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(train_data.normalized)

tokenized_train= tokenizer.texts_to_sequences(train_data.normalized)
tokenized_test= tokenizer.texts_to_sequences(test_data.normalized)

In [90]:
tokenizer.word_index.keys().__len__()


674

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_padded= pad_sequences(tokenized_train, maxlen= 30, padding= "pre")
test_padded= pad_sequences(tokenized_test, maxlen= 30, padding= "pre")

In [0]:
#this function transform final processed text (columns padded) into 3D matrix (samples, steps, unique_words)
#matrix contents one hot encoded words. Encoding was done for each step and based on unique words

def transform_x(data, tokenizer):
    output_shape= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()]
    results= np.zeros(output_shape)
    
    for i in range(data.shape[0]):
        for ii in range(data.shape[1]):
            results[i, ii, data[i,ii]-1]= 1
    return results

xtr_transformed= transform_x(train_padded, tokenizer)
xts_transformed= transform_x(test_padded, tokenizer)


In [93]:
xts_transformed.shape

(1436, 30, 674)

In [0]:
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as CC
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2


class LSTMModel(object):
    
    def build_model(self, input_dim, output_shape, steps, dropout_rate, kernel_regularizer, bias_regularizer):
        input_layer= Input(shape= (steps, input_dim))
        
        #make lstm_layer
        lstm= LSTM(units= steps)(input_layer)
        dense_1= Dense(output_shape, kernel_initializer= he_uniform(),
                       bias_initializer= "zeros", 
                       kernel_regularizer= l2(l= kernel_regularizer),
                       bias_regularizer= l2(l= bias_regularizer))(lstm)
        x= BatchNormalization()(dense_1)
        x= relu(x)
        x= Dropout(rate= dropout_rate)(x)
        o= Dense(output_shape, kernel_initializer= glorot_uniform(),
                 bias_initializer= "zeros", 
                 kernel_regularizer= l2(l= kernel_regularizer), 
                 bias_regularizer= l2(l= bias_regularizer))(dense_1)
        o= BatchNormalization()(o)
        output= softmax(o, axis= 1)
        
        loss= CC()
        metrics= AUC()
        optimizer= Adam()
        self.model= Model(inputs= [input_layer], outputs= [output])
        self.model.compile(optimizer= optimizer, loss= loss, metrics= [metrics])
        
        
    def train(self, x, y, validation_split, epochs):
        self.model.fit(x, y, validation_split= validation_split, epochs= epochs)
        
    def predict(self, x):
        return self.model.predict(x)

In [0]:
steps= xtr_transformed.shape[1]
dim= xtr_transformed.shape[2]
output_shape= ytr_encoded.shape[1]

model= LSTMModel()
model.build_model(input_dim= dim,
                  output_shape= output_shape,
                  steps= steps, 
                  dropout_rate= 0.5, 
                  bias_regularizer= 0.3, 
                  kernel_regularizer= 0.3)

In [96]:
model.train(xtr_transformed, ytr_encoded,
           0.2, 60)

Train on 7000 samples, validate on 1750 samples
Epoch 1/60
7000/7000 [==============================] - 10s 1ms/sample - loss: 12.2737 - auc_3: 0.8092 - val_loss: 5.9325 - val_auc_3: 0.9907
Epoch 2/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 3.5727 - auc_3: 0.9507 - val_loss: 1.8083 - val_auc_3: 0.9984
Epoch 3/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.8411 - auc_3: 0.9749 - val_loss: 1.5156 - val_auc_3: 0.9869
Epoch 4/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.3720 - auc_3: 0.9856 - val_loss: 1.1716 - val_auc_3: 0.9932
Epoch 5/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.1265 - auc_3: 0.9900 - val_loss: 0.8199 - val_auc_3: 0.9953
Epoch 6/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.9439 - auc_3: 0.9923 - val_loss: 0.8103 - val_auc_3: 0.9970
Epoch 7/60
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.7984 - auc_3: 0.9945 - val_los

In [100]:
from sklearn.metrics import classification_report

prediction= y_encoder.inverse_transform(model.predict(xtr_transformed))
print(classification_report(train_data.target, prediction))

                                          precision    recall  f1-score   support

                       atis_abbreviation       0.95      0.99      0.97       147
                           atis_aircraft       0.89      1.00      0.94        81
atis_aircraft#atis_flight#atis_flight_no       0.00      0.00      0.00         1
                            atis_airfare       1.00      1.00      1.00       423
           atis_airfare#atis_flight_time       0.00      0.00      0.00         1
                            atis_airline       0.97      1.00      0.98       157
             atis_airline#atis_flight_no       0.00      0.00      0.00         2
                            atis_airport       0.00      0.00      0.00        20
                           atis_capacity       0.40      0.25      0.31        16
                           atis_cheapest       0.00      0.00      0.00         2
                               atis_city       0.00      0.00      0.00        19
               

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
###test
prediction_test= y_encoder.inverse_transform(model.predict(xts_transformed))
print(classification_report(test_data.target, prediction_test))

                     precision    recall  f1-score   support

  atis_abbreviation       1.00      0.82      0.90        33
      atis_aircraft       0.80      0.89      0.84         9
       atis_airfare       0.92      1.00      0.96        48
       atis_airline       0.95      0.95      0.95        38
        atis_flight       0.99      0.99      0.99      1264
   atis_flight_time       1.00      1.00      1.00         2
atis_ground_service       1.00      1.00      1.00        36
      atis_quantity       0.40      1.00      0.57         6

           accuracy                           0.98      1436
          macro avg       0.88      0.96      0.90      1436
       weighted avg       0.99      0.98      0.98      1436



In [0]:
###############Conclusion
####This is an imbalanced dataset with some of the classes having small number of samples. 
##The resulting poor classification accuracy can be seen in the results.